In [1]:
# pip install faiss-cpu sentence-transformers langchain PyPDF2
!pip install langchain langchain_community langchain_chroma langchain_text_splitters langchain_huggingface sentence-transformers langchain_qdrant qdrant-client langchain_ollama pypdf

In [2]:
#para guardar las librerias requeridas
!pip freeze > requirements.txt

In [3]:
# Step 1: Load and preprocess the PDF
"""Este módulo contiene funciones para cargar documentos PDF utilizando la biblioteca LangChain."""

from langchain_community.document_loaders import PyPDFLoader
from dotenv import load_dotenv

# Cargar las variables de entorno
load_dotenv()

def load_pdf(file_path: str):
    """
    Carga un archivo PDF y devuelve su contenido como documentos procesados.

    Args:
        file_path: Ruta al archivo PDF a cargar.

    Returns:
        docs: Lista de documentos procesados extraídos del PDF.
    """
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    return docs

In [4]:
!pip install pypdf

In [5]:
# Example file path for Brazilian food regulation PDF
#file_path = "/Users/carloszurita/Documents/GIT/practicos-rag/data/biblioteca-de-alimentos.pdf"
file_path = "../Castro_Cofre_Zurita/data/biblioteca-de-alimentos.pdf"

pdf_text = load_pdf(file_path)

In [6]:
"""
# Step 2: Split the document into chunks
def split_text_into_chunks(documents, output_folder: str, chunk_size: int = 1000, chunk_overlap: int = 200) -> list:
    
    Divide el texto de un conjunto de documentos en chunks, guarda cada chunk en la carpeta especificada
    y retorna la lista de chunks.

    Args:
        documents: Lista de documentos procesados (output de PyPDFLoader).
        output_folder: Ruta de la carpeta donde se guardarán los chunks.
        chunk_size: Tamaño máximo de cada chunk en caracteres (por defecto, 1000).
        chunk_overlap: Cantidad de caracteres de solapamiento entre chunks (por defecto, 200).

    Returns:
        chunks: Lista de chunks generados a partir del texto de los documentos.

    import os
    from langchain.text_splitter import RecursiveCharacterTextSplitter

    # Verificar que la carpeta de salida exista, si no, crearla
    os.makedirs(output_folder, exist_ok=True)

    # Concatenar el texto de los documentos
    all_texts = [doc.page_content for doc in documents]  # Extraer texto de cada documento
    concatenated_text = " ".join(all_texts)  # Concatenar en una sola cadena

    # Crear documentos directamente desde el texto
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    documents = text_splitter.create_documents([concatenated_text])

    # Dividir los documentos en chunks
    chunks = text_splitter.split_documents(documents)

    # Guardar cada chunk como un archivo de texto
    for i, chunk in enumerate(chunks):
        chunk_file_path = os.path.join(output_folder, f"chunk_{i + 1}.txt")
        with open(chunk_file_path, "w", encoding="utf-8") as file:
            file.write(chunk.page_content)

    print(f"{len(chunks)} chunks guardados en la carpeta: {output_folder}")
    
    # Retornar los chunks
    return chunks
"""

'\n# Step 2: Split the document into chunks\ndef split_text_into_chunks(documents, output_folder: str, chunk_size: int = 1000, chunk_overlap: int = 200) -> list:\n    \n    Divide el texto de un conjunto de documentos en chunks, guarda cada chunk en la carpeta especificada\n    y retorna la lista de chunks.\n\n    Args:\n        documents: Lista de documentos procesados (output de PyPDFLoader).\n        output_folder: Ruta de la carpeta donde se guardarán los chunks.\n        chunk_size: Tamaño máximo de cada chunk en caracteres (por defecto, 1000).\n        chunk_overlap: Cantidad de caracteres de solapamiento entre chunks (por defecto, 200).\n\n    Returns:\n        chunks: Lista de chunks generados a partir del texto de los documentos.\n\n    import os\n    from langchain.text_splitter import RecursiveCharacterTextSplitter\n\n    # Verificar que la carpeta de salida exista, si no, crearla\n    os.makedirs(output_folder, exist_ok=True)\n\n    # Concatenar el texto de los documentos\n

In [7]:
def generate_chunks(documents, chunk_size: int = 1000, chunk_overlap: int = 200) -> list:
    """
    Divide el texto de un conjunto de documentos en chunks.

    Args:
        documents: Lista de documentos procesados (output de PyPDFLoader).
        chunk_size: Tamaño máximo de cada chunk en caracteres (por defecto, 1000).
        chunk_overlap: Cantidad de caracteres de solapamiento entre chunks (por defecto, 200).

    Returns:
        chunks: Lista de chunks generados a partir del texto de los documentos.
    """
    from langchain.text_splitter import RecursiveCharacterTextSplitter

    # Concatenar el texto de los documentos
    all_texts = [doc.page_content for doc in documents]  # Extraer texto de cada documento
    concatenated_text = " ".join(all_texts)  # Concatenar en una sola cadena

    # Crear documentos directamente desde el texto
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    documents = text_splitter.create_documents([concatenated_text])

    # Dividir los documentos en chunks
    chunks = text_splitter.split_documents(documents)
    return chunks

In [8]:
def save_chunks(chunks: list, output_folder: str):
    """
    Guarda una lista de chunks en archivos de texto individuales dentro de una carpeta.

    Args:
        chunks: Lista de chunks generados.
        output_folder: Ruta de la carpeta donde se guardarán los chunks.

    Returns:
        None
    """
    import os

    # Verificar que la carpeta de salida exista, si no, crearla
    os.makedirs(output_folder, exist_ok=True)

    # Guardar cada chunk como un archivo de texto
    for i, chunk in enumerate(chunks):
        chunk_file_path = os.path.join(output_folder, f"chunk_{i + 1}.txt")
        with open(chunk_file_path, "w", encoding="utf-8") as file:
            file.write(chunk.page_content)

    print(f"{len(chunks)} chunks guardados en la carpeta: {output_folder}")

In [9]:
# Ruta de la carpeta donde se guardarán los chunks
output_folder = "../Castro_Cofre_Zurita/data/chunks/"

# Generar los chunks
chunks = generate_chunks(pdf_text, chunk_size=1000, chunk_overlap=200)

# Guardar los chunks en la carpeta
save_chunks(chunks, output_folder)

69 chunks guardados en la carpeta: ../Castro_Cofre_Zurita/data/chunks/


In [10]:
'''
# Example file path for Brazilian food regulation PDF
# Ruta de la carpeta donde se guardarán los chunks
#output_folder = "/Users/carloszurita/Documents/GIT/practicos-rag/data/chunks/"
output_folder = "../Castro_Cofre_Zurita/data/chunks/"

# Dividir el texto en chunks y guardarlos
chunks=split_text_into_chunks(pdf_text, output_folder)
'''

'\n# Example file path for Brazilian food regulation PDF\n# Ruta de la carpeta donde se guardarán los chunks\n#output_folder = "/Users/carloszurita/Documents/GIT/practicos-rag/data/chunks/"\noutput_folder = "../Castro_Cofre_Zurita/data/chunks/"\n\n# Dividir el texto en chunks y guardarlos\nchunks=split_text_into_chunks(pdf_text, output_folder)\n'

In [11]:
# Step 3: Generate embeddings using Hugging Face
"""Este módulo contiene funciones para generar embeddings a partir de texto utilizando Hugging Face."""

from langchain.embeddings import HuggingFaceEmbeddings

def generate_embeddings(chunks: list, model_name: str = "sentence-transformers/all-MiniLM-L6-v2") -> tuple:
    """
    Genera embeddings a partir de una lista de chunks de texto utilizando un modelo de Hugging Face.

    Args:
        chunks: Lista de textos en formato string para los cuales se generarán los embeddings.
        model_name: Nombre del modelo preentrenado de Hugging Face a utilizar (por defecto, "sentence-transformers/all-MiniLM-L6-v2").

    Returns:
        embeddings: Lista de vectores de embeddings generados para los chunks de texto.
        embedding_model: El modelo de embeddings utilizado.
    """
    # Cargar el modelo de embeddings
    embedding_model = HuggingFaceEmbeddings(model_name=model_name)
    
    # Generar embeddings para los chunks
    embeddings = embedding_model.embed_documents(chunks)
    
    return embeddings, embedding_model

In [12]:
"""Este módulo guarda los embeddings en la carpeta embeddings como archivos JSON."""

import os
import json

def save_embeddings_as_json(embeddings: list, output_folder: str = "data/embeddings"):
    """
    Guarda una lista de embeddings en archivos JSON individuales dentro de una carpeta específica.

    Args:
        embeddings: Lista de vectores de embeddings a guardar.
        output_folder: Ruta de la carpeta donde se guardarán los embeddings (por defecto, "data/embeddings").

    Returns:
        None
    """
    # Crear la carpeta de salida si no existe
    os.makedirs(output_folder, exist_ok=True)

    # Guardar cada embedding en un archivo .json separado
    for i, embedding in enumerate(embeddings):
        file_path = os.path.join(output_folder, f"chunk_{i + 1}_embedding.json")
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump({"embedding": embedding}, f, indent=4)

    print(f"{len(embeddings)} embeddings guardados en formato JSON en la carpeta: {output_folder}")

In [13]:
# Generar embeddings para los chunks
embeddings , embedding_model = generate_embeddings([chunk.page_content for chunk in chunks])

C:\Users\nicol\AppData\Local\Temp\ipykernel_34368\33323915.py:19: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name=model_name)
c:\Users\nicol\OneDrive\MIA\Cursos\SEGUNDO BIMESTRE\PROYECTO APLICADO\PROYECTO\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
output_folder = "../Castro_Cofre_Zurita/data/embeddings"
save_embeddings_as_json(embeddings, output_folder)

69 embeddings guardados en formato JSON en la carpeta: ../Castro_Cofre_Zurita/data/embeddings


In [15]:
# Verificar las dimensiones del embedding
sample_embedding = embedding_model.embed_query("test query")
print(f"Dimensiones del embedding: {len(sample_embedding)}")

Dimensiones del embedding: 384


In [16]:
"""Este módulo contiene funciones para crear y gestionar una base de datos vectorial utilizando Qdrant."""

from dotenv import load_dotenv
from langchain_qdrant import QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

# Cargar las variables de entorno
load_dotenv()

def create_qdrant_vector_store(docs: list, embedding_model, collection_name: str = "demo_collection", path: str = "/tmp/langchain_qdrant", vector_size: int = 384) -> QdrantVectorStore:
    """
    Crea una base de datos vectorial en Qdrant y almacena documentos con sus embeddings.

    Args:
        docs: Lista de documentos (chunks) a almacenar en la base de datos.
        embedding_model: Modelo de embeddings a utilizar para calcular vectores.
        collection_name: Nombre de la colección en Qdrant (por defecto, "demo_collection").
        path: Ruta donde se almacenará la base de datos Qdrant (por defecto, "/tmp/langchain_qdrant").
        vector_size: Tamaño de los vectores de embedding (por defecto, 768).

    Returns:
        qdrant_store: Objeto QdrantVectorStore listo para realizar búsquedas.
    """
    # Inicializar el cliente Qdrant
    client = QdrantClient(path=path)

    # Eliminar la colección existente si ya existe
    client.delete_collection(collection_name=collection_name)

    # Crear una nueva colección en Qdrant
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE),
    )

    # Crear un QdrantVectorStore
    vector_store = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=embedding_model,
    )

    # Agregar documentos a la colección
    vector_store.add_documents(docs)

    print(f"Base de datos vectorial '{collection_name}' creada y {len(docs)} documentos añadidos.")

    return vector_store

In [17]:
def retrieve_documents(query: str, vector_store, k: int = 5) -> list:
    """
    Recupera los documentos más relevantes para una consulta dada utilizando QdrantVectorStore.

    Args:
        query: Texto de la consulta.
        vector_store: Objeto QdrantVectorStore donde se encuentran almacenados los documentos.
        k: Número de documentos relevantes a recuperar (por defecto, 5).

    Returns:
        docs: Lista de los documentos más relevantes encontrados para la consulta.
    """
    # Realizar la búsqueda de similitud en el vector store
    docs = vector_store.similarity_search(query, k=k)

    return docs

In [18]:
# Crear base de datos vectorial
vector_store = create_qdrant_vector_store(chunks, embedding_model)

# Realizar una búsqueda
query = "What are the labeling regulations for food products in Brazil?"
results = retrieve_documents(query, vector_store)

ValueError: could not broadcast input array from shape (384,) into shape (1536,)

In [ ]:
print(results)

NameError: name 'results' is not defined

In [ ]:
# Example query 2
query = "Quais são as normas para lactantes no Brasil?"
results = retrieve_documents(query,vector_store)

# Display results
for i, doc in enumerate(results):
    print(f"Result {i + 1}:\n{doc}\n")

Result 1:
page_content='Lei 11.265/2006 – Regulamenta a comercialização de alimentos para lactentes e crianças de primeira infância 
e a de produtos de puericultura correlatos. 
 Alterada por: 
Lei 11.474/2007  – Altera a Lei no 10.188, de 12 de fevereiro de 2001, que cria o Programa de 
Arrendamento Residencial, institui o arrendamento residencial com opção de compra, e a Lei no 
11.265, de 3 de janeiro de 2006, que regulamenta a comercialização de alimentos para lactentes e 
crianças de primeira infância e a de produtos de puericultura correlatos, e dá outras providências. 
Ato relacionado: 
Decreto 9.579/2018 – Consolida atos normativos editados pelo Poder Executivo federal que dispõem 
sobre a temática do lactente, da criança e do adolescente e do aprendiz, e sobre o Conselho Nacional 
dos Direitos da Criança e do Adolescente, o Fundo Nacional para a Criança e o Adolescente e os 
programas federais da criança e do adolescente.' metadata={'_id': '83243dd0c138437aa29487984d0ee2e9', '

In [ ]:
# Example query 
query = "what about alcoholic normas?" #   ""Quais são as normas de bebidas alcoólicas no Brasil?"
results = retrieve_documents(query,vector_store)

# Display results
for i, doc in enumerate(results):
    print(f"Result {i + 1}:\n{doc}\n")

Result 1:
page_content='BIBLIOTECA DE 
ALIMENTOS 
Atualizada em 06.11.2024 
Coordenação de Processos Regulatórios – CPROR 
Assessoria de Melhoria da Qualidade Regulatória – ASREG 
Gabinete do Diretor-Presidente 
 
AGENDA REGULATÓRIA 
Ciclo Bienal 
2024-2025   
 
APRESENTAÇÃO 
 
 As Bibliotecas são documentos que reúnem todas as normas vigentes de determinado 
macrotema, divididos por temas. O objetivo é facilitar o acesso e a compreensão do Estoque 
Regulatório ao público interno e externo, bem como aprimorar o processo de elaboração e 
revisão das normativas. 
 
 
Não deixe de consultar também a Biblioteca de Temas Transversais, que abrange assuntos 
aplicados a todos os macrotemas, tais como: Autorização de Funcionamento de Empresa 
(AFE), Certificação de Boas Práticas de Fabricação (CBPF), Taxas de Fiscalização de Vigilância 
Sanitária (TFVS), Peticionamento de Recursos, etc. 
 
  
No setor de alimentos, a Anvisa coordena, supervisiona e controla as atividades de registro, inspeção,